In [1]:
# imports
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import urllib.request
from PIL import Image

In [2]:
# site request
url = "https://pokemondb.net/pokedex/national"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find_all("div", class_="infocard")

In [3]:
# loops to collect data for image url, pokemon name, and types
poke_list = []

# for generations 1-8
for i in range(0, 905):
    results[i] = str(results[i])
    dex_no = i +1
    if results[i].split("<")[6].split(" ")[5].replace("src=", "").replace("/>", "").startswith("\"https:"):
        image = results[i].split("<")[6].split(" ")[5].replace("src=", "").replace("/>", "")
    else:
        image = results[i].split("<")[6].split(" ")[6].replace("src=", "").replace("/>", "")
    image = image.strip('\"')
    name = results[i].split("<")[14].split(">")[1]
    type1 = results[i].split("<")[18].split(">")[1]
    if results[i].split("<")[20].split(">")[1] is None:
        pass
    else:
        type2 = results[i].split("<")[20].split(">")[1]
    key_names = ["dex_no", "image", "name", "type1", "type2"]
    values = [dex_no, image, name, type1, type2]
    dict_i = dict(zip(key_names, values))
    poke_list.append(dict_i)

# for generation 9
for i in range(905, len(results)):
    results[i] = str(results[i])
    dex_no = i +1
    if results[i].split("<")[4].split(" ")[5].replace("src=", "").replace("/>", "").startswith("\"https:"):
        image = results[i].split("<")[4].split(" ")[5].replace("src=", "").replace("/>", "")
    else:
        image = results[i].split("<")[4].split(" ")[6].replace("src=", "").replace("/>", "")
    image = image.strip('\"')
    name = results[i].split("<")[11].split(">")[1]
    type1 = results[i].split("<")[15].split(">")[1]
    if results[i].split("<")[17].split(">")[1] is None:
        pass
    else:
        type2 = results[i].split("<")[17].split(">")[1]
    key_names = ["dex_no", "image", "name", "type1", "type2"]
    values = [dex_no, image, name, type1, type2]
    dict_i = dict(zip(key_names, values))
    poke_list.append(dict_i)

In [4]:
# convert list to dataframe

pokedex = pd.DataFrame.from_dict(poke_list)
pokedex

,dex_no,image,name,type1,type2
0,1,https://img.pokemondb.net/sprites/home/normal/...,Bulbasaur,Grass,Poison
1,2,https://img.pokemondb.net/sprites/home/normal/...,Ivysaur,Grass,Poison
2,3,https://img.pokemondb.net/sprites/home/normal/...,Venusaur,Grass,Poison
3,4,https://img.pokemondb.net/sprites/home/normal/...,Charmander,Fire,
4,5,https://img.pokemondb.net/sprites/home/normal/...,Charmeleon,Fire,
...,...,...,...,...,...
1005,1006,https://img.pokemondb.net/sprites/scarlet-viol...,Iron Valiant,Fairy,Fighting
1006,1007,https://img.pokemondb.net/sprites/scarlet-viol...,Koraidon,Fighting,Dragon
1007,1008,https://img.pokemondb.net/sprites/scarlet-viol...,Miraidon,Electric,Dragon
1008,1009,https://img.pokemondb.net/sprites/scarlet-viol...,Walking Wake,Water,Dragon


In [5]:
# rename images
pokedex["image_name"] = pokedex["image"].apply(lambda x: x.split("/")[-1] )
pokedex["image_name"] = "pokedex_" + pokedex["image_name"] 
pokedex

,dex_no,image,name,type1,type2,image_name
0,1,https://img.pokemondb.net/sprites/home/normal/...,Bulbasaur,Grass,Poison,pokedex_bulbasaur.jpg
1,2,https://img.pokemondb.net/sprites/home/normal/...,Ivysaur,Grass,Poison,pokedex_ivysaur.jpg
2,3,https://img.pokemondb.net/sprites/home/normal/...,Venusaur,Grass,Poison,pokedex_venusaur.jpg
3,4,https://img.pokemondb.net/sprites/home/normal/...,Charmander,Fire,,pokedex_charmander.jpg
4,5,https://img.pokemondb.net/sprites/home/normal/...,Charmeleon,Fire,,pokedex_charmeleon.jpg
...,...,...,...,...,...,...
1005,1006,https://img.pokemondb.net/sprites/scarlet-viol...,Iron Valiant,Fairy,Fighting,pokedex_iron-valiant.png
1006,1007,https://img.pokemondb.net/sprites/scarlet-viol...,Koraidon,Fighting,Dragon,pokedex_koraidon.png
1007,1008,https://img.pokemondb.net/sprites/scarlet-viol...,Miraidon,Electric,Dragon,pokedex_miraidon.png
1008,1009,https://img.pokemondb.net/sprites/scarlet-viol...,Walking Wake,Water,Dragon,pokedex_walking-wake.png


In [6]:
# add images to local folder
for index, row in pokedex.iterrows():
    name = row["image_name"]
    url = row["image"]
    response = requests.get(url)
    path = (f"data/images/{name}")
    with open (path, "wb") as file:
        file.write(response.content)

In [7]:
# store dataframe as csv locally
from pathlib import Path  
filepath = Path('data/pokedex.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
pokedex.to_csv(filepath) 